## 01 | Market Segment Vulnerability: Identifying High-Risk Booking Channels

**Business Question:** Which market segments represent the highest risk for the hotel due to elevated cancellation rates?

In [ ]:
WITH filtered_bookings AS (
	SELECT
		hb.market_segment,
		hb.is_canceled
	FROM hotel_bookings hb
	WHERE hb.market_segment <> 'Undefined'
)
SELECT
	fb.market_segment,
	COUNT(*) AS total_bookings,
	ROUND(AVG(fb.is_canceled) * 100.0, 2) AS cancellation_pct
FROM filtered_bookings fb
GROUP BY fb.market_segment
ORDER BY cancellation_pct DESC;

### Visual Insight: Market Segment Risk Distribution

This dashboard-style visualization identifies which market segments contribute most to the hotel's revenue loss. By analyzing these trends through visual analytics, the hotel can implement more targeted cancellation policies, focusing on high-risk channels while prioritizing high-conversion segments.

![Market Segment Cancellation](../images/cancellation_analysis/market_segment_cancellation.PNG)

**Business Analysis:** The data highlights that **Online TA** and **Groups** exhibit the highest volatility, with cancellation rates of **35.37%** and **27.07%** respectively (marked in orange). This disparity suggests a lower level of commitment from third-party intermediaries compared to **Direct** or **Corporate** bookings, which maintain much healthier margins below 15%. To stabilize revenue, the hotel should consider implementing non-refundable deposits or stricter overbooking models specifically for these high-risk segments.

## 02 | Lead Time Impact: Analyzing Booking Windows and Cancellation Risk

**Business Question:** Does booking far in advance increase the probability of cancellation compared to last-minute reservations?

In [ ]:
WITH lead_time_segments AS (
	SELECT
		hb.is_canceled,
		CASE
			WHEN hb.lead_time <= 7 THEN '1. Last minute (0-7d)'
			WHEN hb.lead_time <= 30 THEN '2. Short term (8-30d)'
			WHEN hb.lead_time <= 90 THEN '3. Medium term (31-90d)'
			ELSE '4. Long term (+90d)'
		END AS lead_type
	FROM hotel_bookings hb
)
SELECT
	lts.lead_type,
	COUNT(*) AS total_bookings,
	ROUND(AVG(lts.is_canceled) * 100.0, 2) AS cancellation_pct
FROM lead_time_segments lts
GROUP BY lts.lead_type
ORDER BY lts.lead_type;

### Visual Insight: Lead Time Correlation

This chart analyzes the relationship between the booking window (lead time) and the probability of cancellation. Understanding this behavior through visual analytics allows the hotel to optimize overbooking strategies for reservations made months in advance.

![Lead Time Cancellation Risk](../images/cancellation_analysis/lead_time_cancellation_risk.PNG)

**Business Analysis:** The data confirms a direct correlation between lead time and cancellation probability: as the booking window increases, so does the risk. While "Last minute" bookings (0-7 days) show a low 8.41% cancellation rate, Long term reservations (+90 days) peak at 36.84% (marked in orange). To mitigate this, the hotel should avoid offering flexible cancellation policies for bookings exceeding a 30-day window, instead pivoting towards non-refundable rates or early-payment incentives to secure commitment and stabilize forecasted occupancy.

## 03 | Special Requests vs. Commitment: Do Requests Reduce Cancellation Risk?

**Business Question:** Do guests who make special requests (parking, cribs, etc.) show a lower cancellation rate compared to those who make no requests?

In [ ]:
WITH guest_engagement AS (
	SELECT
		hb.is_canceled,
		CASE
			WHEN hb.required_car_parking_spaces > 0 OR hb.total_of_special_requests > 0 THEN 'With requests'
			ELSE 'No requests'
		END AS engagement_type
	FROM hotel_bookings hb
)
SELECT
	ge.engagement_type,
	COUNT(*) AS total_bookings,
	ROUND(AVG(ge.is_canceled) * 100.0, 2) AS cancellation_pct
FROM guest_engagement ge
GROUP BY ge.engagement_type;

### Visual Insight: Customer Engagement & Commitment

This visualization highlights the correlation between guest requests and booking reliability. The interactive report reveals that specialized service requirements often act as a behavioral signal for confirmed intent, significantly reducing the probability of cancellation.

![Special Requests Impact](../images/cancellation_analysis/special_requests_cancellation_rate.PNG)

**Business Analysis:** The data reveals a significant gap in commitment levels: guests with no special requests show a cancellation rate of **35.75%**, which is **15.48 percentage points higher** than those who engage with the hotel by making requests (**20.27%**). This suggests that the effort of specifying needs, such as parking or cribs, indicates a more finalized travel plan. To capitalize on this, the hotel should implement automated pre-stay engagement emails to encourage guests to personalize their stay, effectively increasing the "psychological cost" of canceling.